In [1]:
import nlopt

from scipy.optimize import OptimizeResult
from six.moves import filter
from functools import partial
from warnings import warn
from re import search

NLOPT_ALGORITHMS_KEYS = list(filter(partial(search, r'^[GL][ND]_'),
                                    dir(nlopt)))
NLOPT_ALGORITHMS = {k: getattr(nlopt, k) for k in NLOPT_ALGORITHMS_KEYS}
NLOPT_MESSAGES = {
    nlopt.SUCCESS: 'Success',
    nlopt.STOPVAL_REACHED: 'Optimization stopped because stopval (above) '
                           'was reached.',
    nlopt.FTOL_REACHED: 'Optimization stopped because ftol_rel or ftol_abs '
                        '(above) was reached.',
    nlopt.XTOL_REACHED: 'Optimization stopped because xtol_rel or xtol_abs '
                        '(above) was reached.',
    nlopt.MAXEVAL_REACHED: 'Optimization stopped because maxeval (above) '
                           'was reached.',
    nlopt.MAXTIME_REACHED: 'Optimization stopped because maxtime (above) '
                           'was reached.',
    nlopt.FAILURE: 'Failure',
    nlopt.INVALID_ARGS: 'Invalid arguments (e.g. lower bounds are bigger '
                        'than upper bounds, an unknown algorithm was '
                        'specified, etcetera).',
    nlopt.OUT_OF_MEMORY: 'Ran out of memory.',
    nlopt.ROUNDOFF_LIMITED: 'Halted because roundoff errors limited progress. '
                            '(In this case, the optimization still typically '
                            'returns a useful result.)',
    nlopt.FORCED_STOP: "Halted because of a forced termination: the user "
                       "called nlopt_force_stop(opt) on the optimization's "
                       "nlopt_opt object opt from the user's objective "
                       "function or constraints."
}


# TODO: argument to specify whether to be stateful
def nlopt_minimize(fun, x0, args=(), method=None, jac=None, bounds=None,
             constraints=[], **options):
    """
    Parameters
    ----------
    fun : callable
        Objective function

    Examples
    --------
    >>> import numpy as np
    >>> from scipy.optimize import rosen, rosen_der
    >>> x0 = [1.3, 0.7, 0.8, 1.9, 1.2]
    >>> res = minimize(rosen, x0, method='ld_lbfgs', jac=rosen_der)
    >>> res.success
    True
    >>> res.message
    'Success'
    >>> np.isclose(res.fun, 0)
    True
    >>> res.x
    array([ 1.,  1.,  1.,  1.,  1.])

    >>> res = minimize(rosen, x0, method='ld_lbfgs', jac=rosen_der,
    ...                ftol_abs=1e-5)
    >>> res.success
    True
    >>> res.message
    'Optimization stopped because ftol_rel or ftol_abs (above) was reached.'

    >>> res = minimize(rosen, x0, method='ld_lbfgs', jac=rosen_der, foo=3)
    Traceback (most recent call last):
        ...
    ValueError: Parameter foo could not be recognized.

    .. todo:: Some sensible way of testing this.

    >>> x0 = np.array([-1., 1.])
    >>> fun = lambda x: - 2*x[0]*x[1] - 2*x[0] + x[0]**2 + 2*x[1]**2
    >>> dfun = lambda x: np.array([2*x[0] - 2*x[1] - 2, - 2*x[0] + 4*x[1]])
    >>> cons = [{'type': 'eq',
    ...           'fun': lambda x: x[0]**3 - x[1],
    ...           'jac': lambda x: np.array([3.*(x[0]**2.), -1.])},
    ...         {'type': 'ineq',
    ...           'fun': lambda x: x[1] - 1,
    ...           'jac': lambda x: np.array([0., 1.])}]
    >>> res = minimize(fun, x0, jac=dfun, method='LD_SLSQP', constraints=cons)
    >>> res.success
    False
    >>> res.message
    'Halted because roundoff errors limited progress. (In this case, the optimization still typically returns a useful result.)'
    >>> res.x.round(2)
    array([ 0.84,  0.6 ])

    >>> cons = [{'type': 'some bogus type',
    ...           'fun': lambda x: x[0]**3 - x[1],
    ...           'jac': lambda x: np.array([3.*(x[0]**2.), -1.])},
    ...         {'type': 'ineq',
    ...           'fun': lambda x: x[1] - 1,
    ...           'jac': lambda x: np.array([0., 1.])}]
    >>> res = minimize(fun, x0, jac=dfun, method='LD_SLSQP', constraints=cons, ftol_abs=1e-20)
    Traceback (most recent call last):
        ...
    ValueError: Constraint type not recognized
    """
    # Create NLopt object
    dim = len(x0)

    # If not a NLopt algorithm enum
    if isinstance(method, str):
        method = get_nlopt_enum(method)

    opt = nlopt.opt(method, dim)

    # Initialize path
    path = []

    # Create NLOpt objective function
    obj_fun = make_nlopt_fun(fun, jac, args, path)
    opt.set_min_objective(obj_fun)

    # Normalize and set parameter bounds
    if bounds:
        lower, upper = zip(*normalize_bounds(bounds))
        opt.set_lower_bounds(lower)
        opt.set_upper_bounds(upper)

    # Equality and Inequality Constraints
    for constr in constraints:

        # Could unpack kwargs here `make_nlopt_fun(**constr)`
        # but we want to support default values
        fun = make_nlopt_fun(fun=constr['fun'],
                             jac=constr.get('jac', False),
                             args=constr.get('args', ()))

        if constr['type'] == 'eq':
            opt.add_equality_constraint(fun)
        elif constr['type'] == 'ineq':
            opt.add_inequality_constraint(fun)
        elif constr['type'] in ('eq_m', 'ineq_m'):
            # TODO: Define '_m' as suffix for now.
            # TODO: Add support for vector/matrix-valued constraints
            raise NotImplementedError('Vector-valued constraints currently '
                                      'not supported.')
        else:
            raise ValueError('Constraint type not recognized')

    # Set other options, e.g. termination criteria
    for option, val in options.items():
        try:
            set_option = getattr(opt, 'set_{option}'.format(option=option))
        except AttributeError:
            raise ValueError('Parameter {option} could not be '
                             'recognized.'.format(option=option))
        else:
            set_option(val)

    # Perform the optimization
    try:
        x = opt.optimize(x0)
    except nlopt.RoundoffLimited:
        # If we encounter a RoundoffLimited exception, simply return last point
        x = path[-1]

    return OptimizeResult(
        x=x,
        fun=opt.last_optimum_value(),
        message=get_nlopt_message(opt.last_optimize_result()),
        success=(opt.last_optimize_result() > 0),
    )


def make_nlopt_fun(fun, jac=True, args=(), path=None):
    """
    Make an NLopt objective function (as specified by the the `NLOpt Python
    interface`_) from SciPy-style objective function.

    The NLOpt objective functions are far less pleasant to work with since they
    are *required* to have side effects because gradient arrays are required to
    be passed-by-reference and modified in-place.

    Parameters
    ----------
    fun : callable
        Objective function

    jac : bool or callable, optional
        Jacobian (gradient) of objective function.

    .. _`NLOpt Python interface`:
       http://ab-initio.mit.edu/wiki/index.php/NLopt_Python_Reference#Objective_function
    """
    def nlopt_fun(x, grad):

        if path is not None:
            path.append(x.copy())

        ret = fun(x, *args)
        grad_temp = None

        if isinstance(ret, tuple):
            val, grad_temp = ret
        else:
            val = ret

        if grad.size > 0:
            if callable(jac):
                grad[:] = jac(x, *args)
            else:
                if bool(jac):
                    if grad_temp is None:
                        warn('Using gradient-based optimization with '
                             'jac=True, but no gradient information is '
                             'available.', RuntimeWarning)
                    else:
                        grad[:] = grad_temp
                else:
                    if grad_temp is not None:
                        warn('Using gradient-based optimization with '
                             'jac=False, the provided gradient information '
                             'is ignored.', RuntimeWarning)

        return val

    return nlopt_fun


def get_nlopt_enum(method_name=None, default=nlopt.LN_BOBYQA):
    """
    Get NLOpt algorithm object by name. If the algorithm is not found,
    defaults to `nlopt.LN_BOBYQA`.

    Notes
    -----

    From http://ab-initio.mit.edu/wiki/index.php/NLopt_Algorithms#Nomenclature:

        Each algorithm in NLopt is identified by a named constant, which
        is passed to the NLopt routines in the various languages in
        order to select a particular algorithm. These constants are
        mostly of the form `NLOPT_{G,L}{N,D}_xxxx`, where G/L denotes
        global/local optimization and N/D denotes derivative-free/
        gradient-based algorithms, respectively.

        For example, the NLOPT_LN_COBYLA constant refers to the COBYLA
        algorithm (described below), which is a local (L)
        derivative-free (N) optimization algorithm.

        Two exceptions are the MLSL and augmented Lagrangian algorithms,
        denoted by NLOPT_G_MLSL and NLOPT_AUGLAG, since whether or not
        they use derivatives (and whether or not they are global, in
        AUGLAG's case) is determined by what subsidiary optimization
        algorithm is specified.

    Equivalent to::

        partial(NLOPT_ALGORITHMS.get, default=nlopt.LN_BOBYQA)

    Examples
    --------
    >>> get_nlopt_enum('LN_NELDERMEAD') == nlopt.LN_NELDERMEAD
    True

    >>> get_nlopt_enum('ln_neldermead') == nlopt.LN_NELDERMEAD
    True

    One is permitted to be cavalier with these method names.

    >>> get_nlopt_enum('ln_NelderMead') == nlopt.LN_NELDERMEAD
    True

    >>> get_nlopt_enum() == nlopt.LN_BOBYQA
    True

    >>> get_nlopt_enum('foobar') == nlopt.LN_BOBYQA
    True

    .. todo:: Exceptional cases (low-priority)

    >>> get_nlopt_enum('G_MLSL') == nlopt.G_MLSL # doctest: +SKIP
    True

    >>> get_nlopt_enum('AUGLAG') == nlopt.AUGLAG # doctest: +SKIP
    True
    """
    if method_name is None:
        method_name = 'LN_BOBYQA'

    try:
        return NLOPT_ALGORITHMS[method_name.upper()]
    except KeyError:
        warn('Method {name} could not be found. Defaulting to '
             '{default}'.format(name=method_name, default=default),
             RuntimeWarning)
        return default


def normalize_bound(bound):
    """
    Examples
    --------
    >>> normalize_bound((2.6, 7.2))
    (2.6, 7.2)

    >>> normalize_bound((None, 7.2))
    (-inf, 7.2)

    >>> normalize_bound((2.6, None))
    (2.6, inf)

    >>> normalize_bound((None, None))
    (-inf, inf)

    This operation is idempotent:

    >>> normalize_bound((-float("inf"), float("inf")))
    (-inf, inf)
    """
    min_, max_ = bound

    if min_ is None:
        min_ = -float('inf')

    if max_ is None:
        max_ = float('inf')

    return min_, max_


def normalize_bounds(bounds=[]):
    """
    Examples
    --------
    >>> bounds = [(2.6, 7.2), (None, 2), (3.14, None), (None, None)]
    >>> list(normalize_bounds(bounds))
    [(2.6, 7.2), (-inf, 2), (3.14, inf), (-inf, inf)]
    """
    return map(normalize_bound, bounds)


def get_nlopt_message(ret_code):
    """
    Notes
    -----
    Identical to ``NLOPT_MESSAGES.get``

    Examples
    --------
    >>> get_nlopt_message(nlopt.SUCCESS)
    'Success'
    >>> get_nlopt_message(nlopt.INVALID_ARGS)
    'Invalid arguments (e.g. lower bounds are bigger than upper bounds, an unknown algorithm was specified, etcetera).'
    """
    return NLOPT_MESSAGES.get(ret_code)

In [2]:
"""[summary]

Raises:
    TypeError: [description]

Returns:
    [type]: [description]
"""
from sys import float_info
from time import time
import warnings
import argparse
import multiprocessing as mp
import numpy as np
# import osmnx as ox
import pickle as pkl
import matplotlib.pyplot as plt
from itertools import combinations as comb
from qiskit import Aer, QuantumCircuit
from qiskit.algorithms.minimum_eigen_solvers import NumPyMinimumEigensolver
from qiskit.algorithms.optimizers import (
                                        ADAM,
                                        CG,
                                        COBYLA,
                                        L_BFGS_B,
                                        NELDER_MEAD,
                                        NFT,
                                        POWELL,
                                        SLSQP,
                                        SPSA,
                                        TNC,
                                        P_BFGS,
                                        BOBYQA
                                        )
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit.utils.quantum_instance import QuantumInstance
from qiskit.opflow import I, X, Y
from qiskit.quantum_info import Statevector
# from qiskit.opflow.primitive_ops import PauliSumOp
# from generate_problem import import_map
from generate_qubo import convert_qubo
from generate_qubo import main as generate_qubo

import QAOAEx


warnings.filterwarnings('ignore')


#Custom Optimizer Classes
from typing import Optional

from scipy.optimize import minimize, basinhopping
from qiskit.algorithms.optimizers.optimizer import Optimizer, OptimizerSupportLevel
from skopt import gp_minimize
from pybobyqa import solve


class Basin(Optimizer):

    _OPTIONS = ['maxiter', 'disp', 'ftol', 'eps']

    # pylint: disable=unused-argument
    def __init__(self,
                 maxiter: int = 100,
                 disp: bool = False,
                 ftol: float = 1e-06,
                 tol: Optional[float] = None,
                 eps: float = 1.4901161193847656e-08) -> None:
        """
        Args:
            maxiter: Maximum number of iterations.
            disp: Set to True to print convergence messages.
            ftol: Precision goal for the value of f in the stopping criterion.
            tol: Tolerance for termination.
            eps: Step size used for numerical approximation of the Jacobian.
        """
        super().__init__()
        for k, v in list(locals().items()):
            if k in self._OPTIONS:
                self._options[k] = v
        self._tol = tol

    def get_support_level(self):
        """ Return support level dictionary """
        return {
            'gradient': OptimizerSupportLevel.supported,
            'bounds': OptimizerSupportLevel.supported,
            'initial_point': OptimizerSupportLevel.required
        }

    def optimize(self, num_vars, objective_function, gradient_function=None,
                 variable_bounds=None, initial_point=None):
        super().optimize(num_vars, objective_function,
                         gradient_function, variable_bounds, initial_point)

        if gradient_function is None and self._max_evals_grouped > 1:
            epsilon = self._options['eps']
            gradient_function = Optimizer.wrap_function(Optimizer.gradient_num_diff,
                                                        (objective_function, epsilon,
                                                         self._max_evals_grouped))
        minimizer_kwargs = {"method": "slsqp", "jac":gradient_function, "tol":self._tol, "bounds":variable_bounds}
        res = basinhopping(objective_function, 
                           initial_point,
                           niter=150,
                           T=0.0,
                           stepsize=0.2*np.pi,
                           minimizer_kwargs=minimizer_kwargs,
                           take_step=None,
                           accept_test=None,
                           callback=None,
                           interval=50,
                           disp=True,
                           niter_success=50,
                           seed=None)
        return res.x, res.fun, res.nfev


class Global_Optimizer(Optimizer):

    _OPTIONS = ['maxiter', 'disp', 'ftol', 'eps']

    # pylint: disable=unused-argument
    def __init__(self,
                 maxiter: int = 100,
                 disp: bool = False,
                 ftol: float = 1e-06,
                 tol: Optional[float] = None,
                 eps: float = 1.4901161193847656e-08,
                n_calls: int=100) -> None:
        """Iteration No: 1 started. Evaluating function at random point.
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 0.2034
Function value obtained: 0
        Args:
            maxiter: Maximum number of iterations.
            disp: Set to True to print convergence messages.
            ftol: Precision goal for the value of f in the stopping criterion.
            tol: Tolerance for termination.
            eps: Step size used for numerical approximation of the Jacobian.
        """
        super().__init__()
        self.n_calls = n_calls
        for k, v in list(locals().items()):
            if k in self._OPTIONS:
                self._options[k] = v
        self._tol = tol

    def get_support_level(self):
        """ Return support level dictionary """
        return {
            'gradient': OptimizerSupportLevel.supported,
            'bounds': OptimizerSupportLevel.supported,
            'initial_point': OptimizerSupportLevel.required
        }

    def optimize(self, num_vars, objective_function, gradient_function=None,
                 variable_bounds=None, initial_point=None):
        super().optimize(num_vars, objective_function,
                         gradient_function, variable_bounds, initial_point)

        if gradient_function is None and self._max_evals_grouped > 1:
            epsilon = self._options['eps']
            gradient_function = Optimizer.wrap_function(Optimizer.gradient_num_diff,
                                                        (objective_function, epsilon,
                                                         self._max_evals_grouped))
        minimizer_kwargs = {"method": "slsqp", "jac":gradient_function, "tol":self._tol, "bounds":variable_bounds}
#         res = basinhopping(objective_function, initial_point, niter=100, T=1.0, stepsize=0.05*np.pi, minimizer_kwargs=None, take_step=None, accept_test=None, callback=None, interval=50, disp=False, niter_success=None, seed=None)
#         res = gp_minimize(objective_function,
#                   dimensions = [(-2*np.pi, 2*np.pi)]*num_vars,      # the bounds on each dimension of x
#                   acq_func="gp_hedge",      # the acquisition function
#                 acq_optimizer = "lbfgs",
#                 n_restarts_optimizer = 10,
#                   n_calls=self.n_calls,         # the number of evaluations of f
#                   n_initial_points = 10,  # the number of random initialization points
#                   noise="gaussian",       # the noise
#                   random_state=1234, # the random seed
#                 verbose = False,
#                  n_jobs = -1)   
#       
#         print(, np.asarray(variable_bounds).shape )
        res = solve(objective_function, initial_point, bounds=np.asarray(variable_bounds).transpose(), npt=None,
            rhobeg=None, rhoend=1e-8, maxfun=None, nsamples=None,
            user_params=None, objfun_has_noise=False,
            seek_global_minimum=True,
            scaling_within_bounds=False,
            do_logging=True, print_progress=False)

        return res.x, res.f, res.nf
    
class NLOPT_Optimizer(Optimizer):

    _OPTIONS = ['maxiter', 'disp', 'ftol', 'eps']

    # pylint: disable=unused-argument
    def __init__(self,
                 maxiter: int = 100,
                 disp: bool = False,
                 ftol: float = 1e-06,
                 tol: Optional[float] = None,
                 eps: float = 1.4901161193847656e-08,
                n_calls: int=100) -> None:
        """Iteration No: 1 started. Evaluating function at random point.
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 0.2034
Function value obtained: 0
        Args:
            maxiter: Maximum number of iterations.
            disp: Set to True to print convergence messages.
            ftol: Precision goal for the value of f in the stopping criterion.
            tol: Tolerance for termination.
            eps: Step size used for numerical approximation of the Jacobian.
        """
        super().__init__()
        self.n_calls = n_calls
        for k, v in list(locals().items()):
            if k in self._OPTIONS:
                self._options[k] = v
        self._tol = tol

    def get_support_level(self):
        """ Return support level dictionary """
        return {
            'gradient': OptimizerSupportLevel.supported,
            'bounds': OptimizerSupportLevel.supported,
            'initial_point': OptimizerSupportLevel.required
        }

    def optimize(self, num_vars, objective_function, gradient_function=None,
                 variable_bounds=None, initial_point=None):
        super().optimize(num_vars, objective_function,
                         gradient_function, variable_bounds, initial_point)

        if gradient_function is None and self._max_evals_grouped > 1:
            epsilon = self._options['eps']
            gradient_function = Optimizer.wrap_function(Optimizer.gradient_num_diff,
                                                        (objective_function, epsilon,
                                                         self._max_evals_grouped))
#         minimizer_kwargs = {"method": "slsqp", "jac":gradient_function, "tol":self._tol, "bounds":variable_bounds}
#         res = basinhopping(objective_function, initial_point, niter=100, T=1.0, stepsize=0.05*np.pi, minimizer_kwargs=None, take_step=None, accept_test=None, callback=None, interval=50, disp=False, niter_success=None, seed=None)
#         res = gp_minimize(objective_function,
#                   dimensions = [(-2*np.pi, 2*np.pi)]*num_vars,      # the bounds on each dimension of x
#                   acq_func="gp_hedge",      # the acquisition function
#                 acq_optimizer = "lbfgs",
#                 n_restarts_optimizer = 10,
#                   n_calls=self.n_calls,         # the number of evaluations of f
#                   n_initial_points = 10,  # the number of random initialization points
#                   noise="gaussian",       # the noise
#                   random_state=1234, # the random seed
#                 verbose = False,
#                  n_jobs = -1)
        res = nlopt_minimize(objective_function, initial_point, method=nlopt.LN_BOBYQA, jac=gradient_function, ftol_abs=1e-32, ftol_rel=1e-32, xtol_abs = 1e-32, xtol_rel = 1e-32, maxtime = 120)
        print(res.message)
        
        return res.x, res.fun, -1

def main(raw_args = None):
    """[summary]

    Args:
        raw_args ([type], optional): [description]. Defaults to None.
    """
    start = time()
    args = parse(raw_args)
    qubo_prob_s_s = []
    qubo_no = args["no_samples"]
#     #Generate and save valid qubos
#     for qubo_no in range(args["no_samples"]):
#         print("__"*50, "\nQUBO NO: {}\n".format(qubo_no), "__"*50)
#         qubo, max_coeff, operator, offset, routes = generate_valid_qubo(args)
# #         print(operator)
#         with open('qubos_{}_car_{}_routes/qubo_{}.pkl'.format(args["no_cars"], args["no_routes"], qubo_no), 'wb') as f:
#             pkl.dump([qubo, max_coeff, operator, offset, routes],f)

    #Load generated qubo_no
    with open('qubos_{}_car_{}_routes/qubo_{}.pkl'.format(args["no_cars"], args["no_routes"], qubo_no), 'rb') as f:
        qubo, max_coeff, operator, offset, routes = pkl.load(f)
    print("__"*50, "\nQUBO NO: {}\n".format(qubo_no), "__"*50)
    classical_result = solve_classically(qubo)   
    x_arr = classical_result.x
    x_str = arr_to_str(x_arr)
    no_qubits = len(x_arr)

    sort_values = get_costs(qubo, no_qubits)

#         print("_"*50)
#         print("10 lowest states:")
#         avg = 0
#         for i in range(27):
#             print(sort_values[i])
#             avg += sort_values[i][1]
#         print("_"*50)
#         print("Avg: {}".format(avg/27))
    ground_energy = sort_values[0][1]
    x_s = [x_str]
    for i in range(1,10):
        if np.round(sort_values[i][1], 4) == np.round(ground_energy, 4):
            print("Other ground state(s) found: '{}'".format(sort_values[i][0]))
            x_s.append(sort_values[i][0])

    # # Visualise
    # if args["visual"]:
    #     graph = import_map('melbourne.pkl')
    #     visualise_solution(graph, routes)

    #Solve QAOA from QUBO with valid solution
    # print(qubo, operator)
    # no_couplings = count_coupling_terms(operator)
    # print("Number of couplings: {}".format(no_couplings))
    print("Solving with QAOA...")
    no_shots = 10000
    backend = Aer.get_backend('statevector_simulator')
    quantum_instance = QuantumInstance(backend, shots = no_shots)

    #Optimizers available from QISKIT - as chosen from parsed argument.
    optimizers = {"ADAM":ADAM(),
    "CG":CG(),
    "COBYLA":COBYLA(),
    "L_BFGS_B":L_BFGS_B(),
    "NELDER_MEAD":NELDER_MEAD(),
    "NFT":NFT(),
    "POWELL":POWELL(),
    "SLSQP":SLSQP(),
    "SPSA":SPSA(),
    "TNC":TNC(),
    "P_BFGS": P_BFGS(),
    "BOBYQA": NLOPT_Optimizer(),
    "GLOBAL": Global_Optimizer(),
    "Basin": Basin()}
    optimizer = optimizers[args["optimizer"]]
    # print("_"*50,"\n"+optimizer.__class__.__name__)
    # print("_"*50)

    quantum_instance = QuantumInstance(backend)

    # for p in range(1, args["p_max"]+1):
        # print("p = {}".format(p))
    prob_s_s = []
    p=1
    print("p=1")
    points = [[0,0]] + [ [ 2.0*np.pi*(np.random.rand() - 0.5) for _ in range(2) ] for _ in range(args["no_restarts"]) ]
#     qc, w_one = construct_initial_state(args)
#     print(qc.draw(fold=200))
    qc = None

#         mixer_terms = [(I ^ left) ^ X ^ (I ^ (args["no_routes"]*args["no_cars"] - left - 1))
#                            for left in range(args["no_routes"]*args["no_cars"])]
#         mixer = sum(mixer_terms)
#     mixer = n_qbit_mixer(qc = qc, n = args["no_routes"]*args["no_cars"])
    mixer = None
#     mixer = mixer_one.copy()
#     for _ in range(0,args["no_cars"]-1):
#         mixer = mixer.tensor(mixer_one)
#     print(mixer.draw(fold=200))
    exp_vals = []
    results = []
    prob_s_2 = []
    for r, point in enumerate(points):
        qaoa_results, _, circ = Fourier_QAOA(operator, quantum_instance, optimizer, p, point, initial_state = qc, mixer = mixer)
        results.append((qaoa_results, circ))
        exp_val = qaoa_results.eigenvalue*max_coeff + offset
        exp_vals.append(exp_val)
        prob_s = 0
        for string in x_s:
            prob_s += qaoa_results.eigenstate[string] if string in qaoa_results.eigenstate else 0
        print("Restart_no: {}, Exp_val: {}, Prob_s: {}".format(r, exp_val, prob_s))
        prob_s_2.append(prob_s)
    minim_index = np.argmin(exp_vals)
    optimal_qaoa_result, circ = results[minim_index]
    print(circ.draw(fold = 150))
    minim_exp_val = exp_vals[minim_index]
    optimal_prob_s = prob_s_2[minim_index]
    optimal_fourier_point = np.array(optimal_qaoa_result.optimal_point)
    print("Minimum: {}, prob_s: {}, point: {}".format(minim_exp_val, optimal_prob_s, QAOAEx.convert_from_fourier_point(optimal_fourier_point, len(optimal_fourier_point))))
    prob_s_s.append(optimal_prob_s)
    minim_exp_val_B = minim_exp_val
    optimal_fourier_point_B = optimal_fourier_point.copy()

    for p in range(2, args["p_max"]+1):
        print("p={}".format(p))
        # if args["optimizer"] == 'SLSQP':
        #     optimizer.set_options(maxiter = )
        # elif args["optimizer"] == 'L_BFGS_B' or 'P_BFGS':
        #     optimizer.set_options(maxfun = 1000*p)
        # print("Optimizer options: {}".format(optimizer.setting))
        next_fourier_point = QAOAEx.convert_from_fourier_point(optimal_fourier_point, 2*p)
        next_fourier_point_B = QAOAEx.convert_from_fourier_point(optimal_fourier_point_B, 2*p)
# #         next_fourier_point = interp_point(next_fourier_point)
# #         next_fourier_point_B = interp_point(next_fourier_point_B)
        next_fourier_point = QAOAEx.convert_to_fourier_point(next_fourier_point, 2*p)
        next_fourier_point_B = QAOAEx.convert_to_fourier_point(next_fourier_point_B, 2*p)

        # next_fourier_point[0:p-1] = optimal_fourier_point[0:p-1]
        # next_fourier_point[p:2*p-1] = optimal_fourier_point[p-1:2*p-2]

        # next_fourier_point_B[0:p-1] = optimal_fourier_point_B[0:p-1]
        # next_fourier_point_B[p:2*p-1] = optimal_fourier_point_B[p-1:2*p-2]

        qaoa_results, _, circ = Fourier_QAOA(operator, quantum_instance, optimizer, p, next_fourier_point, qc, mixer = mixer)
#         print(circ.draw(fold=150))
        optimal_fourier_point = np.array(qaoa_results.optimal_point)
        minim_exp_val = qaoa_results.eigenvalue*max_coeff + offset
#         # print("Length_L:",len(optimal_fourier_point))
        optimal_prob_s = 0
        for string in x_s:
            optimal_prob_s += qaoa_results.eigenstate[string] if string in qaoa_results.eigenstate else 0
        print("Minim_exp_val_L: {}, prob_s_L: {}".format(minim_exp_val, optimal_prob_s))
#         optimal_prob_s_B = optimal_prob_s
#         t1 = time()
#         t = 0
#         results = []
#         exp_vals = []
#         while True:
#             t+=1
#             penalty = 0.6
#             perturbed_points = generate_points(next_fourier_point_B, 10, penalty)
#             for point in perturbed_points:
#                 qaoa_results_B, _, _ = Fourier_QAOA(operator, quantum_instance, optimizer, p, point, qc, mixer = mixer)
#                 exp_val = qaoa_results_B.eigenvalue*max_coeff + offset
#                 results.append(qaoa_results_B)
#                 exp_vals.append(exp_val)
#             exp_val_index = np.argmin(exp_vals)
#             minim_exp_val_B_2 = exp_vals[exp_val_index]
#             qaoa_results_B_2 = results[exp_val_index]
#             optimal_fourier_point_B_2 = np.array(qaoa_results_B_2.optimal_point)
#             optimal_prob_s_B_2 = 0
#             for string in x_s:
#                 optimal_prob_s_B_2 += qaoa_results_B_2.eigenstate[string] if string in qaoa_results_B_2.eigenstate else 0
#             print("Comparison (t={}): B_next: {}, B_before: {}".format(t, minim_exp_val_B_2, minim_exp_val_B))
#             if minim_exp_val_B_2 < minim_exp_val_B or t>=2 and minim_exp_val_B_2 == minim_exp_val_B or t==4:
#                 minim_exp_val_B = minim_exp_val_B_2
#                 optimal_prob_s_B = optimal_prob_s_B_2
#                 optimal_fourier_point_B = optimal_fourier_point_B_2
#                 # print("\a")
#                 break
#             else:
#                 penalty *= 1.2 if t <= 4 else 1
#         if minim_exp_val_B >= minim_exp_val:
#             optimal_fourier_point_B = optimal_fourier_point.copy()
#         t2 = time()
#         print("Time for intialising with perturbed values {} no. of times: {}".format(t, t2-t1))
#         print("Minim_exp_val_B: {}, prob_s_B: {}".format(minim_exp_val_B, optimal_prob_s_B))
#         if minim_exp_val <= minim_exp_val_B:
#             prob_s_s.append(optimal_prob_s)
#         else:
#             prob_s_s.append(optimal_prob_s_B)
        prob_s_s.append(optimal_prob_s)
#         print("Length_B:", len(optimal_fourier_point_B))
        print("Optimal_L:", QAOAEx.convert_from_fourier_point(optimal_fourier_point, len(optimal_fourier_point)))
#         point_B = QAOAEx.convert_from_fourier_point(optimal_fourier_point_B, len(optimal_fourier_point_B))
#         print("Optimal_B: {}, length: {}".format(point_B, len(point_B)))

    print(prob_s_s)

    with open('results_reg/{}cars{}routes_qubo{}_{}.csv'.format(args["no_cars"], args["no_routes"], args["no_samples"], args["optimizer"]), 'w') as f:
        np.savetxt(f, prob_s_s, delimiter=',')
    finish = time()
    print("Time Taken: {}".format(finish - start))


def Fourier_QAOA(operator, quantum_instance, optimizer, p, initial_fourier_point, initial_state=None, mixer = None, qubo = None):
    qaoa_instance = QAOAEx.QAOACustom(quantum_instance = quantum_instance,
                                        reps = p,
                                        force_shots = False,
                                        optimizer = optimizer,
                                        qaoa_name = "example_qaoa",
                                        initial_state = initial_state,
                                        mixer = mixer,
                                        include_custom = True,
                                        max_evals_grouped = 1
                                        )
    bounds = [(-2*np.pi, 2*np.pi)]*2*p
    qaoa_instance.set_parameterise_point_for_energy_evaluation(QAOAEx.convert_from_fourier_point)
    qaoa_results = qaoa_instance.solve(operator, initial_fourier_point, bounds = bounds)
    optimal_parameterised_point = qaoa_instance.latest_parameterised_point
    
    if not isinstance(qaoa_results.eigenstate, dict):
        qaoa_results.eigenstate = Statevector(qaoa_results.eigenstate).probabilities_dict()
    qc = qaoa_instance.get_optimal_circuit()
    return qaoa_results, optimal_parameterised_point, qc

def generate_points(point, no_perturb, penalty):
    points = [point.copy() for _ in range(no_perturb+1)]
    for r in range(1,no_perturb+1):
        for i in range(len(point)):
            if point[i] == 0:
                noise = np.random.choice([-0.05*np.pi, 0, 0.05*np.pi])
                points[r][i] += penalty*noise
            else:
                noise = np.random.normal(0, point[i]**2)
                points[r][i] += penalty*noise
    return points

def parse(raw_args):
    """Parse inputs for no_cars (number of cars),
                        no_routes (number of routes)

    Args:
        raw_args (list): list of strings describing the inputs e.g. ["-N 3", "-R 3"]

    Returns:
        dict: A dictionary of the inputs as values, and the name of variables as the keys
    """
    optimizer_choices = ["ADAM", "CG", "COBYLA", "L_BFGS_B",\
                        "NELDER_MEAD", "NFT", "POWELL",\
                         "SLSQP", "SPSA", "TNC", "P_BFGS", "BOBYQA", "GLOBAL", "Basin"]
    parser = argparse.ArgumentParser()
    required_named = parser.add_argument_group('Required arguments')
    required_named.add_argument("--no_cars", "-N",
                                required = True,
                                help="Set the number of cars",
                                type = int
                                )
    required_named.add_argument("--no_routes", "-R",
                                required = True,
                                help="Set the number of routes for each car",
                                type = int
                                )
    required_named.add_argument("--penalty_multiplier", "-P",
                                required = True,
                                help="Set the penalty multiplier for QUBO constraint",
                                type = float
                                )
    required_named.add_argument("--optimizer", "-O",
                                required = True,
                                help = "Choose from Qiskit's optimizers",
                                type = str,
                                choices=optimizer_choices
                                )
    required_named.add_argument("--p_max", "-M",
                                required = True,
                                help = "Set maximum number of layers for QAOA",
                                type = int
                                )
    required_named.add_argument("--no_restarts", "-T", required = True,
                                help = "Set number of restarts for QAOA",
                                type = int
                                )
    required_named.add_argument("--no_samples", "-S", required = True,
                                help = "Set number of samples/qubos with given no_cars no_routes",
                                type=int
                                )
    parser.add_argument("--visual",
                        "-V", default = False,
                        help="Activate routes visualisation with '-V' ",
                        action="store_true"
                        )
    args = parser.parse_args(raw_args)
    args = vars(args)
    return args

def solve_classically(qubo):
    """[summary]

    Args:
        qubo ([type]): [description]

    Returns:
        [type]: [description]
    """
    exact_mes = NumPyMinimumEigensolver()
    exact = MinimumEigenOptimizer(exact_mes)
    exact_result = exact.solve(qubo)
    print(exact_result)
    return exact_result

def arr_to_str(x_arr):
    """[summary]

    Args:
        x_arr ([type]): [description]

    Returns:
        [type]: [description]
    """
    x_arr = x_arr[::-1]
    string = ''.join(str(int(x_i)) for x_i in x_arr)
    return string

def filter_solutions(results, qaoa_dict, no_cars):
    """[summary]

    Args:
        results ([type]): [description]
        qaoa_dict ([type]): [description]
        no_cars ([type]): [description]

    Returns:
        [type]: [description]
    """
    routes = []
    variables = []
    for var_route, var_value in zip( results.items(), qaoa_dict.items() ):
        if var_route[0] == var_value[0] and int(var_value[1]) == 1 \
            and var_route[0] not in variables:
            routes.append(var_route[1])
            variables.append(var_route[0])
        elif var_route[0] != var_value[0]:
            print("Error, function returned non-matching variables")
            return None
        elif var_route[0] in variables:
            print("Error: Solution found two routes for one car")
            return None
    if len(variables) != no_cars:
        print("Error: At least one car did not have a valid route")
        return None
    return routes

def eval_cost(x_str_arr, qubo_problem, no_qubits):
    """[summary]

    Args:
        x ([type]): [description]
        qubo_problem ([type]): [description]
        no_qubits ([type]): [description]
        max_coeff ([type]): [description]
        offset ([type]): [description]

    Returns:
        [type]: [description]
    """
    if isinstance(x_str_arr, np.ndarray):
        cost = qubo_problem.objective.evaluate([int(x_str_arr[i]) for i in range(no_qubits)])
    else:
        cost = qubo_problem.objective.evaluate([int(x_str_arr[no_qubits-1-i]) \
                                                for i in range(no_qubits)]
                                            )
    return x_str_arr, cost

# def visualise_solution(graph, routes):
#     """[summary]

#     Args:ca
#         graph ([type]): [description]
#         routes ([type]): [description]

#     Returns:
#         [type]: [description]
#     """
#     colors = np.array(["r", "y", "b", "g", "w"]*10)[0:len(routes)]
#     fig, axes = ox.plot_graph_routes(graph,
#                                     routes,
#                                     route_colors=colors,
#                                     route_linewidth=4,
#                                     node_size=10
#                                     )
#     return fig, axes

def get_costs(qubo, no_qubits):
    """[summary]

    Args:
        qubo ([type]): [description]
        no_qubits ([type]): [description]

    Returns:
        [type]: [description]
    """
    # values = ['{}'.format()]
    cpus = mp.cpu_count()
    pool = mp.Pool(cpus-1)
    params = ( ('0'*(no_qubits-len('{0:b}'.format(k)))+'{0:b}'.format(k),
            qubo,
            no_qubits
            ) for k in range(2**no_qubits))
    values = pool.starmap(eval_cost, params)
    values = dict(values)
    pool.close()
    pool.join()
    sort_values = sorted(values.items(), key=lambda x: x[1])
    return sort_values

def check_solutions(routes):
    """[summary]

    Args:
        routes ([type]): [description]

    Raises:
        TypeError: [description]
    """
    try:
        len(routes)
        print("This is a valid solution")
    except TypeError as typ:
        raise TypeError("This is not a valid solution") from typ

def count_coupling_terms(operator):
    """[summary]

    Args:
        operator ([type]): [description]
    """
    count = 0
    for op_i in operator.to_pauli_op().oplist:
        op_str = str(op_i.primitive)
        if op_str.count('Z') == 2:
            count += 1
    return count

def generate_valid_qubo(args):
    while True: #Generate valid QUBO
        try:
            #Generate a QUBO
            if args["visual"] is True:
                qubo, max_coeff, operator, offset, linear, quadratic, results = \
                    generate_qubo( [
                                    "-N "+str(args["no_cars"]),
                                    "-R "+str(args["no_routes"]),
                                    "-P "+str(args["penalty_multiplier"]),
                                    "-V"
                                    ]
                                )
            else:
                qubo, max_coeff, operator, offset, linear, quadratic, results = \
                    generate_qubo( [
                                    "-N "+str(args["no_cars"]),
                                    "-R "+str(args["no_routes"]),
                                    "-P "+str(args["penalty_multiplier"])
                                    ]
                                )
            #Classically solve generated QUBO
            classical_result = solve_classically(qubo)

            #Check solution
            variables_dict = classical_result.variables_dict
            routes = filter_solutions(results, variables_dict, args["no_cars"])
            check_solutions(routes)

            #If valid solution, save the solution.
            x_arr = classical_result.x
            x_str = arr_to_str(x_arr)

            break #break loop if valid solution, else increase penalty.

        except TypeError:
            #QUBO has invalid solution, start again with increased penalty.
            print("Starting with new qubo")
            args["penalty_multiplier"] += 0.05
    return qubo, max_coeff, operator, offset, routes

def interp_point(optimal_point):
    """Method to interpolate to next point from the optimal point found from the previous layer   
    
    Args:
        optimal_point (np.array): Optimal point from previous layer
    
    Returns:
        point (list): the informed next point
    """
    optimal_point = list(optimal_point)
    p = int(len(optimal_point)/2)
    gammas = [0]+optimal_point[0:p]+[0]
    betas = [0]+optimal_point[p:2*p]+[0]
    interp_gammas = [0]+gammas
    interp_betas = [0]+betas    
    for i in range(1,p+2):
        interp_gammas[i] = gammas[i-1]*(i-1)/p+gammas[i]*(p+1-i)/p
        interp_betas[i] = betas[i-1]*(i-1)/p+betas[i]*(p+1-i)/p    
    
    point = interp_gammas[1:p+2] + interp_betas[1:p+2]

    return point

def construct_initial_state(args):
    """[summary]

    Args:
        args ([type]): [description]

    Returns:
        [type]: [description]
    """
    R = args["no_routes"]
    N = args["no_cars"]
    #Using gates
    w_one = QuantumCircuit(R)
    for r in range(0,R-1):
        if r == 0:
            w_one.ry(2*np.arccos(1/np.sqrt(R-r)),r)
        elif r != R-1:
            w_one.cry(2*np.arccos(1/np.sqrt(R-r)), r-1, r)
    for r in range(1,R):
        w_one.cx(R-r-1,R-r)
    w_one.x(0)

    # #Using Initial Instruction
    # w_one = QuantumCircuit(R)
    # probs = [0 for _ in range(2**R)]
    # for i in range(R):
    #     probs[2**i] += 1/np.sqrt(R)
    # w_one.initialize(probs)

    #Tensor product for N cars
    initial_state = w_one.copy()
    for _ in range(0,N-1):
        initial_state = initial_state.tensor(w_one)
    initial_state.name = 'U1'
    return initial_state, w_one

def n_qbit_mixer(qc, n):
    #Using circuit mixer
    from qiskit.circuit.parameter import Parameter
    t = Parameter('t')
    mixer = QuantumCircuit(n)
    mixer.append(qc.inverse(), range(n))
    mixer.rz(t,range(n))
    mixer.append(qc, range(n))
#     mixer.h(range(n))
    
#     mixer.rz(t, range(n))
#     for r in range(3):
#         for i,j in comb(range(3), 2):
#             i+= 3*r
#             j+= 3*r
#             mixer.cx(i, j)
#             mixer.rz(t, j)
#             mixer.cx(i, j)
#     for i in range(n-1):
#             mixer.cx(i, i+1)
#             mixer.rz(t, i+1)
#             mixer.cx(i, i+1)     
    
#     mixer.h(range(n))

   #Using Xi, YiYj
#     paulis = []
#     for (i,j) in comb(list(range(n)),2):
#         X_iX_j = (I^(n-j-1))^X^(I^(j-i-1))^X^(I^i)
#         paulis.append(X_iX_j)
#         Y_iY_j = (I^(n-j-1))^Y^(I^(j-i-1))^Y^(I^i)
#         paulis.append(Y_iY_j)
#     mixer = sum(paulis)
    
#     paulis = []
#     for i in range(n):
#         paulis.append( (I ^ (n-i-1)) ^ X ^ (I^i))
#         paulis.append( (I ^ (n-i-1)) ^ Y ^ (I^i))
#     for r in range(3):
#         for i,j in comb(range(3), 2):
#             i += 3*r
#             j += 3*r
#             paulis.append( (I^(n-j-1))^X^(I^(j-i-1))^X^(I^i) )            
#             paulis.append( (I^(n-j-1))^Y^(I^(j-i-1))^Y^(I^i) )
            
    mixer.name = 'Exp_Hm'
    return mixer

# def n_qbit_mixer(n):
#     from qiskit.circuit.parameter import Parameter
# #     from qiskit.circuit.parametervector import ParameterVector
    
#     t = Parameter('t')
#     qc = QuantumCircuit(n)
#     qc_two = QuantumCircuit(2)
#     qc_two.sdg(1)
#     qc_two.cx(1,0)
#     qc_two.ry(-2*t,1)
#     qc_two.cx(0,1)
#     qc_two.ry(2*t,1)
#     qc_two.cx(1,0)
#     qc_two.s(0)
#     if n%2:
#         for i in range(int((n-1)/2)):
#             qc.append(qc_two,(2*i,2*i+1))
#             qc.swap(2*i,2*i+1)
#         for i in range(int((n-1)/2)):
#             qc.append(qc_two,(2*i+1,2*i+2))
#             qc.swap(2*i+1,2*i+2)
#         for i in range(int((n-1)/2)):
#             qc.append(qc_two,(2*i,2*i+1))
#             qc.swap(2*i,2*i+1)
#         for i in range(int((n-1)/2)):
#             qc.append(qc_two,(2*i+1,2*i+2))
#     else:
#         for i in range(int((n)/2)):
#             qc.append(qc_two,(2*i,2*i+1))
#             qc.swap(2*i,2*i+1)
#         for i in range(1,int((n)/2)):
#             qc.append(qc_two,(2*i-1,2*i))
#             qc.swap(2*i-1,2*i)
#         for i in range(int((n)/2)):
#             qc.append(qc_two,(2*i,2*i+1))
#             qc.swap(2*i,2*i+1)
#         for i in range(1,int((n)/2)):
#             qc.append(qc_two,(2*i-1,2*i))
    
#     return qc

# main()
# main(["-N 3", "-R 3", "-P 1.5", "-OSLSQP", "-M 10", "-T 2", "-S 1"])


# operator, offset = quadratic_program.to_ising()

# initial_point = [0.40784, 0.73974, -0.53411, -0.28296]
# print()
# print("Solving QAOA...")
# qaoa_results = qaoa_instance.solve(operator, initial_point)

# qaoa_results_eigenstate = qaoa_results.eigenstate
# print("optimal_value:", qaoa_results.optimal_value)
# print("optimal_parameters:", qaoa_results.optimal_parameters)
# print("optimal_point:", qaoa_results.optimal_point)
# print("optimizer_evals:", qaoa_results.optimizer_evals)

# solutions = qaoa_instance.get_optimal_solutions_from_statevector(qaoa_results_eigenstate, quadratic_program)
# print_qaoa_solutions(solutions)

In [3]:
for i in range(20):
    main(["-N 3", "-R 3", "-P 0.5", "-OBOBYQA", "-M 6", "-T 0", "-S {}".format(i)])

____________________________________________________________________________________________________ 
QUBO NO: 0
 ____________________________________________________________________________________________________
optimal function value: 4.4326419999999995
optimal value: [1. 0. 0. 1. 0. 0. 1. 0. 0.]
status: SUCCESS
Solving with QAOA...
p=1
Optimization stopped because xtol_rel or xtol_abs (above) was reached.
Restart_no: 0, Exp_val: 16.470257149368543, Prob_s: 0.0005474356293861703
     ┌───┐┌───┐┌──────────────────────┐┌───┐┌───┐┌──────────────────────┐┌───┐          ┌───┐          ┌────────────────────────┐»
q_0: ┤ H ├┤ X ├┤ RZ(1.96093295582208) ├┤ X ├┤ X ├┤ RZ(1.96093295582208) ├┤ X ├──────────┤ X ├──────────┤ RZ(0.0108043698507662) ├»
     ├───┤└─┬─┘└──────────────────────┘└─┬─┘└─┬─┘└──────────────────────┘└─┬─┘          └─┬─┘          └─────────┬───┬──────────┘»
q_1: ┤ H ├──■────────────────────────────■────┼────────────────────────────┼──────────────┼──────────────────────┤ X ├─

KeyboardInterrupt: 

In [41]:
"""[summary]

Raises:
    TypeError: [description]

Returns:
    [type]: [description]
"""
from sys import float_info
from time import time
import warnings
import argparse
import multiprocessing as mp
import numpy as np
# import osmnx as ox
import pickle as pkl
import matplotlib.pyplot as plt
from itertools import combinations as comb
from qiskit import Aer, QuantumCircuit
from qiskit.algorithms.minimum_eigen_solvers import NumPyMinimumEigensolver
from qiskit.algorithms.optimizers import (
                                        ADAM,
                                        CG,
                                        COBYLA,
                                        L_BFGS_B,
                                        NELDER_MEAD,
                                        NFT,
                                        POWELL,
                                        SLSQP,
                                        SPSA,
                                        TNC,
                                        P_BFGS,
                                        BOBYQA
                                        )
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit.utils.quantum_instance import QuantumInstance
from qiskit.opflow import I, X, Y
from qiskit.quantum_info import Statevector
# from qiskit.opflow.primitive_ops import PauliSumOp
# from generate_problem import import_map
from generate_qubo import convert_qubo
from generate_qubo import main as generate_qubo

import QAOAEx


warnings.filterwarnings('ignore')


#Custom Optimizer Classes
from typing import Optional

from scipy.optimize import minimize, basinhopping
from qiskit.algorithms.optimizers.optimizer import Optimizer, OptimizerSupportLevel
from skopt import gp_minimize
from pybobyqa import solve


class Basin(Optimizer):

    _OPTIONS = ['maxiter', 'disp', 'ftol', 'eps']

    # pylint: disable=unused-argument
    def __init__(self,
                 maxiter: int = 100,
                 disp: bool = False,
                 ftol: float = 1e-06,
                 tol: Optional[float] = None,
                 eps: float = 1.4901161193847656e-08) -> None:
        """
        Args:
            maxiter: Maximum number of iterations.
            disp: Set to True to print convergence messages.
            ftol: Precision goal for the value of f in the stopping criterion.
            tol: Tolerance for termination.
            eps: Step size used for numerical approximation of the Jacobian.
        """
        super().__init__()
        for k, v in list(locals().items()):
            if k in self._OPTIONS:
                self._options[k] = v
        self._tol = tol

    def get_support_level(self):
        """ Return support level dictionary """
        return {
            'gradient': OptimizerSupportLevel.supported,
            'bounds': OptimizerSupportLevel.supported,
            'initial_point': OptimizerSupportLevel.required
        }

    def optimize(self, num_vars, objective_function, gradient_function=None,
                 variable_bounds=None, initial_point=None):
        super().optimize(num_vars, objective_function,
                         gradient_function, variable_bounds, initial_point)

        if gradient_function is None and self._max_evals_grouped > 1:
            epsilon = self._options['eps']
            gradient_function = Optimizer.wrap_function(Optimizer.gradient_num_diff,
                                                        (objective_function, epsilon,
                                                         self._max_evals_grouped))
        minimizer_kwargs = {"method": "slsqp", "jac":gradient_function, "tol":self._tol, "bounds":variable_bounds}
        res = basinhopping(objective_function, 
                           initial_point,
                           niter=150,
                           T=0.0,
                           stepsize=0.2*np.pi,
                           minimizer_kwargs=minimizer_kwargs,
                           take_step=None,
                           accept_test=None,
                           callback=None,
                           interval=50,
                           disp=True,
                           niter_success=50,
                           seed=None)
        return res.x, res.fun, res.nfev


class Global_Optimizer(Optimizer):

    _OPTIONS = ['maxiter', 'disp', 'ftol', 'eps']

    # pylint: disable=unused-argument
    def __init__(self,
                 maxiter: int = 100,
                 disp: bool = False,
                 ftol: float = 1e-06,
                 tol: Optional[float] = None,
                 eps: float = 1.4901161193847656e-08,
                n_calls: int=100) -> None:
        """Iteration No: 1 started. Evaluating function at random point.
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 0.2034
Function value obtained: 0
        Args:
            maxiter: Maximum number of iterations.
            disp: Set to True to print convergence messages.
            ftol: Precision goal for the value of f in the stopping criterion.
            tol: Tolerance for termination.
            eps: Step size used for numerical approximation of the Jacobian.
        """
        super().__init__()
        self.n_calls = n_calls
        for k, v in list(locals().items()):
            if k in self._OPTIONS:
                self._options[k] = v
        self._tol = tol

    def get_support_level(self):
        """ Return support level dictionary """
        return {
            'gradient': OptimizerSupportLevel.supported,
            'bounds': OptimizerSupportLevel.supported,
            'initial_point': OptimizerSupportLevel.required
        }

    def optimize(self, num_vars, objective_function, gradient_function=None,
                 variable_bounds=None, initial_point=None):
        super().optimize(num_vars, objective_function,
                         gradient_function, variable_bounds, initial_point)

        if gradient_function is None and self._max_evals_grouped > 1:
            epsilon = self._options['eps']
            gradient_function = Optimizer.wrap_function(Optimizer.gradient_num_diff,
                                                        (objective_function, epsilon,
                                                         self._max_evals_grouped))
        minimizer_kwargs = {"method": "slsqp", "jac":gradient_function, "tol":self._tol, "bounds":variable_bounds}
#         res = basinhopping(objective_function, initial_point, niter=100, T=1.0, stepsize=0.05*np.pi, minimizer_kwargs=None, take_step=None, accept_test=None, callback=None, interval=50, disp=False, niter_success=None, seed=None)
#         res = gp_minimize(objective_function,
#                   dimensions = [(-2*np.pi, 2*np.pi)]*num_vars,      # the bounds on each dimension of x
#                   acq_func="gp_hedge",      # the acquisition function
#                 acq_optimizer = "lbfgs",
#                 n_restarts_optimizer = 10,
#                   n_calls=self.n_calls,         # the number of evaluations of f
#                   n_initial_points = 10,  # the number of random initialization points
#                   noise="gaussian",       # the noise
#                   random_state=1234, # the random seed
#                 verbose = False,
#                  n_jobs = -1)   
#       
#         print(, np.asarray(variable_bounds).shape )
        res = solve(objective_function, initial_point, bounds=np.asarray(variable_bounds).transpose(), npt=None,
            rhobeg=None, rhoend=1e-8, maxfun=None, nsamples=None,
            user_params=None, objfun_has_noise=False,
            seek_global_minimum=True,
            scaling_within_bounds=False,
            do_logging=True, print_progress=False)

        return res.x, res.f, res.nf
    
class NLOPT_Optimizer(Optimizer):

    _OPTIONS = ['maxiter', 'disp', 'ftol', 'eps']

    # pylint: disable=unused-argument
    def __init__(self,
                 maxiter: int = 100,
                 disp: bool = False,
                 ftol: float = 1e-06,
                 tol: Optional[float] = None,
                 eps: float = 1.4901161193847656e-08,
                n_calls: int=100) -> None:
        """Iteration No: 1 started. Evaluating function at random point.
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 0.2034
Function value obtained: 0
        Args:
            maxiter: Maximum number of iterations.
            disp: Set to True to print convergence messages.
            ftol: Precision goal for the value of f in the stopping criterion.
            tol: Tolerance for termination.
            eps: Step size used for numerical approximation of the Jacobian.
        """
        super().__init__()
        self.n_calls = n_calls
        for k, v in list(locals().items()):
            if k in self._OPTIONS:
                self._options[k] = v
        self._tol = tol

    def get_support_level(self):
        """ Return support level dictionary """
        return {
            'gradient': OptimizerSupportLevel.supported,
            'bounds': OptimizerSupportLevel.supported,
            'initial_point': OptimizerSupportLevel.required
        }

    def optimize(self, num_vars, objective_function, gradient_function=None,
                 variable_bounds=None, initial_point=None):
        super().optimize(num_vars, objective_function,
                         gradient_function, variable_bounds, initial_point)

        if gradient_function is None and self._max_evals_grouped > 1:
            epsilon = self._options['eps']
            gradient_function = Optimizer.wrap_function(Optimizer.gradient_num_diff,
                                                        (objective_function, epsilon,
                                                         self._max_evals_grouped))
#         minimizer_kwargs = {"method": "slsqp", "jac":gradient_function, "tol":self._tol, "bounds":variable_bounds}
#         res = basinhopping(objective_function, initial_point, niter=100, T=1.0, stepsize=0.05*np.pi, minimizer_kwargs=None, take_step=None, accept_test=None, callback=None, interval=50, disp=False, niter_success=None, seed=None)
#         res = gp_minimize(objective_function,
#                   dimensions = [(-2*np.pi, 2*np.pi)]*num_vars,      # the bounds on each dimension of x
#                   acq_func="gp_hedge",      # the acquisition function
#                 acq_optimizer = "lbfgs",
#                 n_restarts_optimizer = 10,
#                   n_calls=self.n_calls,         # the number of evaluations of f
#                   n_initial_points = 10,  # the number of random initialization points
#                   noise="gaussian",       # the noise
#                   random_state=1234, # the random seed
#                 verbose = False,
#                  n_jobs = -1)
        res = nlopt_minimize(objective_function, initial_point, method=nlopt.LN_COBYLA, jac=gradient_function, ftol_abs=1e-6)
        
        return res.x, res.fun, -1

def main(raw_args = None):
    """[summary]

    Args:
        raw_args ([type], optional): [description]. Defaults to None.
    """
    start = time()
    args = parse(raw_args)
    qubo_prob_s_s = []
    qubo_no = args["no_samples"]
#     #Generate and save valid qubos
#     for qubo_no in range(args["no_samples"]):
#         print("__"*50, "\nQUBO NO: {}\n".format(qubo_no), "__"*50)
#         qubo, max_coeff, operator, offset, routes = generate_valid_qubo(args)
# #         print(operator)
#         with open('qubos_{}_car_{}_routes/qubo_{}.pkl'.format(args["no_cars"], args["no_routes"], qubo_no), 'wb') as f:
#             pkl.dump([qubo, max_coeff, operator, offset, routes],f)

    #Load generated qubo_no
    with open('qubos_{}_car_{}_routes/qubo_{}.pkl'.format(args["no_cars"], args["no_routes"], qubo_no), 'rb') as f:
        qubo, max_coeff, operator, offset, routes = pkl.load(f)
    print("__"*50, "\nQUBO NO: {}\n".format(qubo_no), "__"*50)
    classical_result = solve_classically(qubo)   
    x_arr = classical_result.x
    x_str = arr_to_str(x_arr)
    no_qubits = len(x_arr)

    sort_values = get_costs(qubo, no_qubits)

#         print("_"*50)
#         print("10 lowest states:")
#         avg = 0
#         for i in range(27):
#             print(sort_values[i])
#             avg += sort_values[i][1]
#         print("_"*50)
#         print("Avg: {}".format(avg/27))
    ground_energy = sort_values[0][1]
    x_s = [x_str]
    for i in range(1,10):
        if np.round(sort_values[i][1], 4) == np.round(ground_energy, 4):
            print("Other ground state(s) found: '{}'".format(sort_values[i][0]))
            x_s.append(sort_values[i][0])

    # # Visualise
    # if args["visual"]:
    #     graph = import_map('melbourne.pkl')
    #     visualise_solution(graph, routes)

    #Solve QAOA from QUBO with valid solution
    # print(qubo, operator)
    # no_couplings = count_coupling_terms(operator)
    # print("Number of couplings: {}".format(no_couplings))
    print("Solving with QAOA...")
    no_shots = 10000
    backend = Aer.get_backend('statevector_simulator')
    quantum_instance = QuantumInstance(backend, shots = no_shots)

    #Optimizers available from QISKIT - as chosen from parsed argument.
    optimizers = {"ADAM":ADAM(),
    "CG":CG(),
    "COBYLA":COBYLA(),
    "L_BFGS_B":L_BFGS_B(),
    "NELDER_MEAD":NELDER_MEAD(),
    "NFT":NFT(),
    "POWELL":POWELL(),
    "SLSQP":SLSQP(),
    "SPSA":SPSA(),
    "TNC":TNC(),
    "P_BFGS": P_BFGS(),
    "BOBYQA": NLOPT_Optimizer(),
    "GLOBAL": Global_Optimizer(),
    "Basin": Basin()}
    optimizer = optimizers[args["optimizer"]]
    # print("_"*50,"\n"+optimizer.__class__.__name__)
    # print("_"*50)

    quantum_instance = QuantumInstance(backend)

    # for p in range(1, args["p_max"]+1):
        # print("p = {}".format(p))
    prob_s_s = []
    p=1
    print("p=1")
    points = [[0,0]] + [ [ 2.0*np.pi*(np.random.rand() - 0.5) for _ in range(2) ] for _ in range(args["no_restarts"]) ]
    qc, w_one = construct_initial_state(args)
#     print(qc.draw(fold=200))
#     qc = None

#         mixer_terms = [(I ^ left) ^ X ^ (I ^ (args["no_routes"]*args["no_cars"] - left - 1))
#                            for left in range(args["no_routes"]*args["no_cars"])]
#         mixer = sum(mixer_terms)
    mixer = n_qbit_mixer(qc = qc, n = args["no_routes"]*args["no_cars"])
#     mixer = None
#     mixer = mixer_one.copy()
#     for _ in range(0,args["no_cars"]-1):
#         mixer = mixer.tensor(mixer_one)
#     print(mixer.draw(fold=200))
    exp_vals = []
    results = []
    prob_s_2 = []
    for r, point in enumerate(points):
        qaoa_results, _, circ = Fourier_QAOA(operator, quantum_instance, optimizer, p, point, initial_state = qc, mixer = mixer)
        results.append((qaoa_results, circ))
        exp_val = qaoa_results.eigenvalue*max_coeff + offset
        exp_vals.append(exp_val)
        prob_s = 0
        for string in x_s:
            prob_s += qaoa_results.eigenstate[string] if string in qaoa_results.eigenstate else 0
        print("Restart_no: {}, Exp_val: {}, Prob_s: {}".format(r, exp_val, prob_s))
        prob_s_2.append(prob_s)
    minim_index = np.argmin(exp_vals)
    optimal_qaoa_result, circ = results[minim_index]
    print(circ.draw(fold = 150))
    minim_exp_val = exp_vals[minim_index]
    optimal_prob_s = prob_s_2[minim_index]
    optimal_fourier_point = np.array(optimal_qaoa_result.optimal_point)
    print("Minimum: {}, prob_s: {}, point: {}".format(minim_exp_val, optimal_prob_s, QAOAEx.convert_from_fourier_point(optimal_fourier_point, len(optimal_fourier_point))))
    prob_s_s.append(optimal_prob_s)
    minim_exp_val_B = minim_exp_val
    optimal_fourier_point_B = optimal_fourier_point.copy()

    for p in range(2, args["p_max"]+1):
        print("p={}".format(p))
        # if args["optimizer"] == 'SLSQP':
        #     optimizer.set_options(maxiter = )
        # elif args["optimizer"] == 'L_BFGS_B' or 'P_BFGS':
        #     optimizer.set_options(maxfun = 1000*p)
        # print("Optimizer options: {}".format(optimizer.setting))
        next_fourier_point = QAOAEx.convert_from_fourier_point(optimal_fourier_point, 2*p)
        next_fourier_point_B = QAOAEx.convert_from_fourier_point(optimal_fourier_point_B, 2*p)
# #         next_fourier_point = interp_point(next_fourier_point)
# #         next_fourier_point_B = interp_point(next_fourier_point_B)
        next_fourier_point = QAOAEx.convert_to_fourier_point(next_fourier_point, 2*p)
        next_fourier_point_B = QAOAEx.convert_to_fourier_point(next_fourier_point_B, 2*p)

        # next_fourier_point[0:p-1] = optimal_fourier_point[0:p-1]
        # next_fourier_point[p:2*p-1] = optimal_fourier_point[p-1:2*p-2]

        # next_fourier_point_B[0:p-1] = optimal_fourier_point_B[0:p-1]
        # next_fourier_point_B[p:2*p-1] = optimal_fourier_point_B[p-1:2*p-2]

        qaoa_results, _, circ = Fourier_QAOA(operator, quantum_instance, optimizer, p, next_fourier_point, qc, mixer = mixer)
#         print(circ.draw(fold=150))
        optimal_fourier_point = np.array(qaoa_results.optimal_point)
        minim_exp_val = qaoa_results.eigenvalue*max_coeff + offset
#         # print("Length_L:",len(optimal_fourier_point))
        optimal_prob_s = 0
        for string in x_s:
            optimal_prob_s += qaoa_results.eigenstate[string] if string in qaoa_results.eigenstate else 0
        print("Minim_exp_val_L: {}, prob_s_L: {}".format(minim_exp_val, optimal_prob_s))
#         optimal_prob_s_B = optimal_prob_s
#         t1 = time()
#         t = 0
#         results = []
#         exp_vals = []
#         while True:
#             t+=1
#             penalty = 0.6
#             perturbed_points = generate_points(next_fourier_point_B, 10, penalty)
#             for point in perturbed_points:
#                 qaoa_results_B, _, _ = Fourier_QAOA(operator, quantum_instance, optimizer, p, point, qc, mixer = mixer)
#                 exp_val = qaoa_results_B.eigenvalue*max_coeff + offset
#                 results.append(qaoa_results_B)
#                 exp_vals.append(exp_val)
#             exp_val_index = np.argmin(exp_vals)
#             minim_exp_val_B_2 = exp_vals[exp_val_index]
#             qaoa_results_B_2 = results[exp_val_index]
#             optimal_fourier_point_B_2 = np.array(qaoa_results_B_2.optimal_point)
#             optimal_prob_s_B_2 = 0
#             for string in x_s:
#                 optimal_prob_s_B_2 += qaoa_results_B_2.eigenstate[string] if string in qaoa_results_B_2.eigenstate else 0
#             print("Comparison (t={}): B_next: {}, B_before: {}".format(t, minim_exp_val_B_2, minim_exp_val_B))
#             if minim_exp_val_B_2 < minim_exp_val_B or t>=2 and minim_exp_val_B_2 == minim_exp_val_B or t==4:
#                 minim_exp_val_B = minim_exp_val_B_2
#                 optimal_prob_s_B = optimal_prob_s_B_2
#                 optimal_fourier_point_B = optimal_fourier_point_B_2
#                 # print("\a")
#                 break
#             else:
#                 penalty *= 1.2 if t <= 4 else 1
#         if minim_exp_val_B >= minim_exp_val:
#             optimal_fourier_point_B = optimal_fourier_point.copy()
#         t2 = time()
#         print("Time for intialising with perturbed values {} no. of times: {}".format(t, t2-t1))
#         print("Minim_exp_val_B: {}, prob_s_B: {}".format(minim_exp_val_B, optimal_prob_s_B))
#         if minim_exp_val <= minim_exp_val_B:
#             prob_s_s.append(optimal_prob_s)
#         else:
#             prob_s_s.append(optimal_prob_s_B)
        prob_s_s.append(optimal_prob_s)
#         print("Length_B:", len(optimal_fourier_point_B))
        print("Optimal_L:", QAOAEx.convert_from_fourier_point(optimal_fourier_point, len(optimal_fourier_point)))
#         point_B = QAOAEx.convert_from_fourier_point(optimal_fourier_point_B, len(optimal_fourier_point_B))
#         print("Optimal_B: {}, length: {}".format(point_B, len(point_B)))

    print(prob_s_s)

    with open('results_changed/{}cars{}routes_qubo{}_{}.csv'.format(args["no_cars"], args["no_routes"], args["no_samples"], args["optimizer"]), 'w') as f:
        np.savetxt(f, prob_s_s, delimiter=',')
    finish = time()
    print("Time Taken: {}".format(finish - start))


def Fourier_QAOA(operator, quantum_instance, optimizer, p, initial_fourier_point, initial_state=None, mixer = None, qubo = None):
    qaoa_instance = QAOAEx.QAOACustom(quantum_instance = quantum_instance,
                                        reps = p,
                                        force_shots = False,
                                        optimizer = optimizer,
                                        qaoa_name = "example_qaoa",
                                        initial_state = initial_state,
                                        mixer = mixer,
                                        include_custom = True,
                                        max_evals_grouped = 1
                                        )
    bounds = [(-2*np.pi, 2*np.pi)]*2*p
    qaoa_instance.set_parameterise_point_for_energy_evaluation(QAOAEx.convert_from_fourier_point)
    qaoa_results = qaoa_instance.solve(operator, initial_fourier_point, bounds = bounds)
    optimal_parameterised_point = qaoa_instance.latest_parameterised_point
    
    if not isinstance(qaoa_results.eigenstate, dict):
        qaoa_results.eigenstate = Statevector(qaoa_results.eigenstate).probabilities_dict()
    qc = qaoa_instance.get_optimal_circuit()
    return qaoa_results, optimal_parameterised_point, qc

def generate_points(point, no_perturb, penalty):
    points = [point.copy() for _ in range(no_perturb+1)]
    for r in range(1,no_perturb+1):
        for i in range(len(point)):
            if point[i] == 0:
                noise = np.random.choice([-0.05*np.pi, 0, 0.05*np.pi])
                points[r][i] += penalty*noise
            else:
                noise = np.random.normal(0, point[i]**2)
                points[r][i] += penalty*noise
    return points

def parse(raw_args):
    """Parse inputs for no_cars (number of cars),
                        no_routes (number of routes)

    Args:
        raw_args (list): list of strings describing the inputs e.g. ["-N 3", "-R 3"]

    Returns:
        dict: A dictionary of the inputs as values, and the name of variables as the keys
    """
    optimizer_choices = ["ADAM", "CG", "COBYLA", "L_BFGS_B",\
                        "NELDER_MEAD", "NFT", "POWELL",\
                         "SLSQP", "SPSA", "TNC", "P_BFGS", "BOBYQA", "GLOBAL", "Basin"]
    parser = argparse.ArgumentParser()
    required_named = parser.add_argument_group('Required arguments')
    required_named.add_argument("--no_cars", "-N",
                                required = True,
                                help="Set the number of cars",
                                type = int
                                )
    required_named.add_argument("--no_routes", "-R",
                                required = True,
                                help="Set the number of routes for each car",
                                type = int
                                )
    required_named.add_argument("--penalty_multiplier", "-P",
                                required = True,
                                help="Set the penalty multiplier for QUBO constraint",
                                type = float
                                )
    required_named.add_argument("--optimizer", "-O",
                                required = True,
                                help = "Choose from Qiskit's optimizers",
                                type = str,
                                choices=optimizer_choices
                                )
    required_named.add_argument("--p_max", "-M",
                                required = True,
                                help = "Set maximum number of layers for QAOA",
                                type = int
                                )
    required_named.add_argument("--no_restarts", "-T", required = True,
                                help = "Set number of restarts for QAOA",
                                type = int
                                )
    required_named.add_argument("--no_samples", "-S", required = True,
                                help = "Set number of samples/qubos with given no_cars no_routes",
                                type=int
                                )
    parser.add_argument("--visual",
                        "-V", default = False,
                        help="Activate routes visualisation with '-V' ",
                        action="store_true"
                        )
    args = parser.parse_args(raw_args)
    args = vars(args)
    return args

def solve_classically(qubo):
    """[summary]

    Args:
        qubo ([type]): [description]

    Returns:
        [type]: [description]
    """
    exact_mes = NumPyMinimumEigensolver()
    exact = MinimumEigenOptimizer(exact_mes)
    exact_result = exact.solve(qubo)
    print(exact_result)
    return exact_result

def arr_to_str(x_arr):
    """[summary]

    Args:
        x_arr ([type]): [description]

    Returns:
        [type]: [description]
    """
    x_arr = x_arr[::-1]
    string = ''.join(str(int(x_i)) for x_i in x_arr)
    return string

def filter_solutions(results, qaoa_dict, no_cars):
    """[summary]

    Args:
        results ([type]): [description]
        qaoa_dict ([type]): [description]
        no_cars ([type]): [description]

    Returns:
        [type]: [description]
    """
    routes = []
    variables = []
    for var_route, var_value in zip( results.items(), qaoa_dict.items() ):
        if var_route[0] == var_value[0] and int(var_value[1]) == 1 \
            and var_route[0] not in variables:
            routes.append(var_route[1])
            variables.append(var_route[0])
        elif var_route[0] != var_value[0]:
            print("Error, function returned non-matching variables")
            return None
        elif var_route[0] in variables:
            print("Error: Solution found two routes for one car")
            return None
    if len(variables) != no_cars:
        print("Error: At least one car did not have a valid route")
        return None
    return routes

def eval_cost(x_str_arr, qubo_problem, no_qubits):
    """[summary]

    Args:
        x ([type]): [description]
        qubo_problem ([type]): [description]
        no_qubits ([type]): [description]
        max_coeff ([type]): [description]
        offset ([type]): [description]

    Returns:
        [type]: [description]
    """
    if isinstance(x_str_arr, np.ndarray):
        cost = qubo_problem.objective.evaluate([int(x_str_arr[i]) for i in range(no_qubits)])
    else:
        cost = qubo_problem.objective.evaluate([int(x_str_arr[no_qubits-1-i]) \
                                                for i in range(no_qubits)]
                                            )
    return x_str_arr, cost

# def visualise_solution(graph, routes):
#     """[summary]

#     Args:ca
#         graph ([type]): [description]
#         routes ([type]): [description]

#     Returns:
#         [type]: [description]
#     """
#     colors = np.array(["r", "y", "b", "g", "w"]*10)[0:len(routes)]
#     fig, axes = ox.plot_graph_routes(graph,
#                                     routes,
#                                     route_colors=colors,
#                                     route_linewidth=4,
#                                     node_size=10
#                                     )
#     return fig, axes

def get_costs(qubo, no_qubits):
    """[summary]

    Args:
        qubo ([type]): [description]
        no_qubits ([type]): [description]

    Returns:
        [type]: [description]
    """
    # values = ['{}'.format()]
    cpus = mp.cpu_count()
    pool = mp.Pool(cpus-1)
    params = ( ('0'*(no_qubits-len('{0:b}'.format(k)))+'{0:b}'.format(k),
            qubo,
            no_qubits
            ) for k in range(2**no_qubits))
    values = pool.starmap(eval_cost, params)
    values = dict(values)
    pool.close()
    pool.join()
    sort_values = sorted(values.items(), key=lambda x: x[1])
    return sort_values

def check_solutions(routes):
    """[summary]

    Args:
        routes ([type]): [description]

    Raises:
        TypeError: [description]
    """
    try:
        len(routes)
        print("This is a valid solution")
    except TypeError as typ:
        raise TypeError("This is not a valid solution") from typ

def count_coupling_terms(operator):
    """[summary]

    Args:
        operator ([type]): [description]
    """
    count = 0
    for op_i in operator.to_pauli_op().oplist:
        op_str = str(op_i.primitive)
        if op_str.count('Z') == 2:
            count += 1
    return count

def generate_valid_qubo(args):
    while True: #Generate valid QUBO
        try:
            #Generate a QUBO
            if args["visual"] is True:
                qubo, max_coeff, operator, offset, linear, quadratic, results = \
                    generate_qubo( [
                                    "-N "+str(args["no_cars"]),
                                    "-R "+str(args["no_routes"]),
                                    "-P "+str(args["penalty_multiplier"]),
                                    "-V"
                                    ]
                                )
            else:
                qubo, max_coeff, operator, offset, linear, quadratic, results = \
                    generate_qubo( [
                                    "-N "+str(args["no_cars"]),
                                    "-R "+str(args["no_routes"]),
                                    "-P "+str(args["penalty_multiplier"])
                                    ]
                                )
            #Classically solve generated QUBO
            classical_result = solve_classically(qubo)

            #Check solution
            variables_dict = classical_result.variables_dict
            routes = filter_solutions(results, variables_dict, args["no_cars"])
            check_solutions(routes)

            #If valid solution, save the solution.
            x_arr = classical_result.x
            x_str = arr_to_str(x_arr)

            break #break loop if valid solution, else increase penalty.

        except TypeError:
            #QUBO has invalid solution, start again with increased penalty.
            print("Starting with new qubo")
            args["penalty_multiplier"] += 0.05
    return qubo, max_coeff, operator, offset, routes

def interp_point(optimal_point):
    """Method to interpolate to next point from the optimal point found from the previous layer   
    
    Args:
        optimal_point (np.array): Optimal point from previous layer
    
    Returns:
        point (list): the informed next point
    """
    optimal_point = list(optimal_point)
    p = int(len(optimal_point)/2)
    gammas = [0]+optimal_point[0:p]+[0]
    betas = [0]+optimal_point[p:2*p]+[0]
    interp_gammas = [0]+gammas
    interp_betas = [0]+betas    
    for i in range(1,p+2):
        interp_gammas[i] = gammas[i-1]*(i-1)/p+gammas[i]*(p+1-i)/p
        interp_betas[i] = betas[i-1]*(i-1)/p+betas[i]*(p+1-i)/p    
    
    point = interp_gammas[1:p+2] + interp_betas[1:p+2]

    return point

def construct_initial_state(args):
    """[summary]

    Args:
        args ([type]): [description]

    Returns:
        [type]: [description]
    """
    R = args["no_routes"]
    N = args["no_cars"]
    #Using gates
    w_one = QuantumCircuit(R)
    for r in range(0,R-1):
        if r == 0:
            w_one.ry(2*np.arccos(1/np.sqrt(R-r)),r)
        elif r != R-1:
            w_one.cry(2*np.arccos(1/np.sqrt(R-r)), r-1, r)
    for r in range(1,R):
        w_one.cx(R-r-1,R-r)
    w_one.x(0)

    # #Using Initial Instruction
    # w_one = QuantumCircuit(R)
    # probs = [0 for _ in range(2**R)]
    # for i in range(R):
    #     probs[2**i] += 1/np.sqrt(R)
    # w_one.initialize(probs)

    #Tensor product for N cars
    initial_state = w_one.copy()
    for _ in range(0,N-1):
        initial_state = initial_state.tensor(w_one)
    initial_state.name = 'U1'
    return initial_state, w_one

def n_qbit_mixer(qc, n):
    #Using circuit mixer
    from qiskit.circuit.parameter import Parameter
    t = Parameter('t')
    mixer = QuantumCircuit(n)
    mixer.append(qc.inverse(), range(n))
    mixer.rz(t,range(n))
    mixer.append(qc, range(n))
#     mixer.h(range(n))
    
#     mixer.rz(t, range(n))
#     for r in range(3):
#         for i,j in comb(range(3), 2):
#             i+= 3*r
#             j+= 3*r
#             mixer.cx(i, j)
#             mixer.rz(t, j)
#             mixer.cx(i, j)
#     for i in range(n-1):
#             mixer.cx(i, i+1)
#             mixer.rz(t, i+1)
#             mixer.cx(i, i+1)     
    
#     mixer.h(range(n))

   #Using Xi, YiYj
#     paulis = []
#     for (i,j) in comb(list(range(n)),2):
#         X_iX_j = (I^(n-j-1))^X^(I^(j-i-1))^X^(I^i)
#         paulis.append(X_iX_j)
#         Y_iY_j = (I^(n-j-1))^Y^(I^(j-i-1))^Y^(I^i)
#         paulis.append(Y_iY_j)
#     mixer = sum(paulis)
    
#     paulis = []
#     for i in range(n):
#         paulis.append( (I ^ (n-i-1)) ^ X ^ (I^i))
#         paulis.append( (I ^ (n-i-1)) ^ Y ^ (I^i))
#     for r in range(3):
#         for i,j in comb(range(3), 2):
#             i += 3*r
#             j += 3*r
#             paulis.append( (I^(n-j-1))^X^(I^(j-i-1))^X^(I^i) )            
#             paulis.append( (I^(n-j-1))^Y^(I^(j-i-1))^Y^(I^i) )
            
    mixer.name = 'Exp_Hm'
    return mixer

# def n_qbit_mixer(n):
#     from qiskit.circuit.parameter import Parameter
# #     from qiskit.circuit.parametervector import ParameterVector
    
#     t = Parameter('t')
#     qc = QuantumCircuit(n)
#     qc_two = QuantumCircuit(2)
#     qc_two.sdg(1)
#     qc_two.cx(1,0)
#     qc_two.ry(-2*t,1)
#     qc_two.cx(0,1)
#     qc_two.ry(2*t,1)
#     qc_two.cx(1,0)
#     qc_two.s(0)
#     if n%2:
#         for i in range(int((n-1)/2)):
#             qc.append(qc_two,(2*i,2*i+1))
#             qc.swap(2*i,2*i+1)
#         for i in range(int((n-1)/2)):
#             qc.append(qc_two,(2*i+1,2*i+2))
#             qc.swap(2*i+1,2*i+2)
#         for i in range(int((n-1)/2)):
#             qc.append(qc_two,(2*i,2*i+1))
#             qc.swap(2*i,2*i+1)
#         for i in range(int((n-1)/2)):
#             qc.append(qc_two,(2*i+1,2*i+2))
#     else:
#         for i in range(int((n)/2)):
#             qc.append(qc_two,(2*i,2*i+1))
#             qc.swap(2*i,2*i+1)
#         for i in range(1,int((n)/2)):
#             qc.append(qc_two,(2*i-1,2*i))
#             qc.swap(2*i-1,2*i)
#         for i in range(int((n)/2)):
#             qc.append(qc_two,(2*i,2*i+1))
#             qc.swap(2*i,2*i+1)
#         for i in range(1,int((n)/2)):
#             qc.append(qc_two,(2*i-1,2*i))
    
#     return qc

# main()
# main(["-N 3", "-R 3", "-P 1.5", "-OSLSQP", "-M 10", "-T 2", "-S 1"])


# operator, offset = quadratic_program.to_ising()

# initial_point = [0.40784, 0.73974, -0.53411, -0.28296]
# print()
# print("Solving QAOA...")
# qaoa_results = qaoa_instance.solve(operator, initial_point)

# qaoa_results_eigenstate = qaoa_results.eigenstate
# print("optimal_value:", qaoa_results.optimal_value)
# print("optimal_parameters:", qaoa_results.optimal_parameters)
# print("optimal_point:", qaoa_results.optimal_point)
# print("optimizer_evals:", qaoa_results.optimizer_evals)

# solutions = qaoa_instance.get_optimal_solutions_from_statevector(qaoa_results_eigenstate, quadratic_program)
# print_qaoa_solutions(solutions)

In [44]:
for i in range(200,201):
    main(["-N 3", "-R 3", "-P 0.5", "-OGLOBAL", "-M 1", "-T 2", "-S {}".format(i)])

____________________________________________________________________________________________________ 
QUBO NO: 200
 ____________________________________________________________________________________________________
optimal function value: 0.3955925940860001
optimal value: [0. 1. 0. 0. 0. 1. 1. 0. 0.]
status: SUCCESS
Other ground state(s) found: '010100001'
Solving with QAOA...
p=1
Restart_no: 0, Exp_val: 0.48905267080207393, Prob_s: 0.13963860106683568
Restart_no: 1, Exp_val: 0.48905267080207127, Prob_s: 0.13963860270063172
Restart_no: 2, Exp_val: 0.4890526708020717, Prob_s: 0.13963860024116348
     ┌────────────┐                     ┌───┐┌───┐┌───────────────────────┐┌───┐┌───┐┌───────────────────────┐┌───┐┌───┐┌────────────────────────┐»
q_0: ┤ RY(1.9106) ├─────■────────────■──┤ X ├┤ X ├┤ RZ(-3.37087215286253) ├┤ X ├┤ X ├┤ RZ(-3.56637353893255) ├┤ X ├┤ X ├┤ RZ(-0.128710477310099) ├»
     └────────────┘┌────┴────┐     ┌─┴─┐└───┘└─┬─┘└───────────────────────┘└─┬─┘└─┬─┘└──────────────

In [46]:
for i in range(200,201):
    main(["-N 3", "-R 3", "-P 0.5", "-OBOBYQA", "-M 2", "-T 2", "-S {}".format(i)])

____________________________________________________________________________________________________ 
QUBO NO: 200
 ____________________________________________________________________________________________________
optimal function value: 0.3955925940860001
optimal value: [0. 1. 0. 0. 0. 1. 1. 0. 0.]
status: SUCCESS
Other ground state(s) found: '010100001'
Solving with QAOA...
p=1
Restart_no: 0, Exp_val: 0.48905274592537884, Prob_s: 0.13960651901535975
Restart_no: 1, Exp_val: 0.4890526727575888, Prob_s: 0.13964466253447427
Restart_no: 2, Exp_val: 0.48905267109197004, Prob_s: 0.13963731412026179
     ┌────────────┐                     ┌───┐┌───┐┌───────────────────────┐┌───┐┌───┐┌───────────────────────┐┌───┐┌───┐┌────────────────────────┐»
q_0: ┤ RY(1.9106) ├─────■────────────■──┤ X ├┤ X ├┤ RZ(-3.37104097217352) ├┤ X ├┤ X ├┤ RZ(-3.56655214930288) ├┤ X ├┤ X ├┤ RZ(-0.128716923361184) ├»
     └────────────┘┌────┴────┐     ┌─┴─┐└───┘└─┬─┘└───────────────────────┘└─┬─┘└─┬─┘└──────────────

In [1]:
# def ansatz(n, reps, operator):
#     from qiskit.circuit.parameter import Parameter
# #     from qiskit.circuit.parametervector import ParameterVector
    
#     t = Parameter('t')
#     s = Parameter('s')
#     qc = QuantumCircuit(n)
#     qc_two = QuantumCircuit(2)
#     qc_two.sdg(1)
#     qc_two.cx(1,0)
#     qc_two.rz(2*s,0)
#     qc_two.ry(-2*t,1)
#     qc_two.cx(0,1)
#     qc_two.ry(2*t,1)
#     qc_two.cx(1,0)
#     qc_two.s(0)
#     for op in operator:
#         op = op.to_pauli_op()
#         op_str = str(op.primitive)
#         op_coeff = op.coeff
#         i_j = [n-i-1 for i in range(len(string)) if op_str.startswith('Z', i)]
#         if op_str.count('Z') == 1:
#             i = i_j[0]
#             qc.rz(2*s*op_coeff, i)
#             qc.rx(2*t, i)
#         if op_str.count('Z') == 2:
#             i, j = i_j
#             qc.append(qc_two.assign_parameters({s:op_coeff*s}), (i,j))
#     final = QuantumCircuit(n)
#     for rep in range(reps):
#         t_p = Parameter('t_{}'.format(rep))
#         s_p = Parameter('s_{}'.format(rep))
#         final.append(qc.assign_parameters({t:t_p, s:s_p}), range(n))
# #     print(qc_two.draw(fold=200))
# #     print(qc.draw(fold=200))
# #     print(final.draw(fold=200))
#     return final

def construct_initial_state(no_cars, no_routes):
    """[summary]

    Args:
        args ([type]): [description]

    Returns:
        [type]: [description]
    """
    R = no_routes
    N = no_cars
    #Using gates
    w_one = QuantumCircuit(R)
    for r in range(0,R-1):
        if r == 0:
            w_one.ry(2*np.arccos(1/np.sqrt(R-r)),r)
        elif r != R-1:
            w_one.cry(2*np.arccos(1/np.sqrt(R-r)), r-1, r)
    for r in range(1,R):
        w_one.cx(R-r-1,R-r)
    w_one.x(0)

    # #Using Initial Instruction
    # w_one = QuantumCircuit(R)
    # probs = [0 for _ in range(2**R)]
    # for i in range(R):
    #     probs[2**i] += 1/np.sqrt(R)
    # w_one.initialize(probs)

    #Tensor product for N cars
    initial_state = w_one.copy()
    for _ in range(0,N-1):
        initial_state = initial_state.tensor(w_one)
    return initial_state, w_one

In [7]:
from qiskit.algorithms import VQE
from qiskit.compiler import transpile

with open('qubos_{}_car_{}_routes/qubo_{}.pkl'.format(3, 3, 3), 'rb') as f:
    qubo, max_coeff, operator, offset, routes = pkl.load(f)

classical_result = solve_classically(qubo)   
x_arr = classical_result.x
x_str = arr_to_str(x_arr)
no_qubits = len(x_arr)

sort_values = get_costs(qubo, no_qubits)

#         print("_"*50)
#         print("10 lowest states:")
#         avg = 0
#         for i in range(27):
#             print(sort_values[i])
#             avg += sort_values[i][1]
#         print("_"*50)
#         print("Avg: {}".format(avg/27))
ground_energy = sort_values[0][1]
x_s = [x_str]
for i in range(1,10):
    if np.round(sort_values[i][1], 4) == np.round(ground_energy, 4):
        print("Other ground state(s) found: '{}'".format(sort_values[i][0]))
        x_s.append(sort_values[i][0])

optimal function value: 3.960833000000001
optimal value: [0. 1. 0. 1. 0. 0. 1. 0. 0.]
status: SUCCESS
Other ground state(s) found: '001010001'


In [712]:
p=1
backend = Aer.get_backend('statevector_simulator')
quantum_instance = QuantumInstance(backend)
initial_state, _ = construct_initial_state(3,3)
qc = ansatz(9,p, operator)
initial_state.append(qc, range(3*3))
initial_state.parameter_bounds = [(-2*np.pi, 2*np.pi)]*initial_state.num_parameters
# print(transpile(initial_state, basis_gates = ['id', 'rx', 'ry', 'rz', 'cx']).draw(fold=200))
vqe = VQE(ansatz = initial_state, optimizer = Global_Optimizer(), include_custom=True, quantum_instance = quantum_instance, initial_point = [np.random.rand() for _ in range(2*p)])
results = vqe.compute_minimum_eigenvalue(operator)
probs = Statevector(results.eigenstate).probabilities_dict()
p_s = 0
for string in x_s:
    p_s += probs[string] if string in probs else 0
print(p_s)
next_point = list(results.optimal_point)

Iteration No: 1 started. Evaluating function at random point.
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 0.0594
Function value obtained: 13.1202
Current minimum: 13.1202
Iteration No: 2 started. Evaluating function at random point.
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 0.0194
Function value obtained: -8.6161
Current minimum: -8.6161
Iteration No: 3 started. Evaluating function at random point.
Iteration No: 3 ended. Evaluation done at random point.
Time taken: 0.0181
Function value obtained: 8.7978
Current minimum: -8.6161
Iteration No: 4 started. Evaluating function at random point.
Iteration No: 4 ended. Evaluation done at random point.
Time taken: 0.0178
Function value obtained: -0.3223
Current minimum: -8.6161
Iteration No: 5 started. Evaluating function at random point.
Iteration No: 5 ended. Evaluation done at random point.
Time taken: 0.0176
Function value obtained: 6.2805
Current minimum: -8.6161
Iteration No: 6 started. Ev

In [ ]:
def ansatz(n, reps, operator):
    from qiskit.circuit.parameter import Parameter
#     from qiskit.circuit.parametervector import ParameterVector
    
    t = Parameter('t')
    s = Parameter('s')
    qc = QuantumCircuit(n)
#     qc_two = QuantumCircuit(2)
#     qc_two.sdg(1)
#     qc_two.cx(1,0)
#     qc_two.rz(2*s,0)
#     qc_two.ry(-2*t,1)
#     qc_two.cx(0,1)
#     qc_two.ry(2*t,1)
#     qc_two.cx(1,0)
#     qc_two.s(0)
    for op in operator:
        op = op.to_pauli_op()
        op_str = str(op.primitive)
        op_coeff = op.coeff
        i_j = [n-i-1 for i in range(len(op_str)) if op_str.startswith('Z', i)]
        if op_str.count('Z') == 1:
            i = i_j[0]
            qc.rz(2*s*op_coeff, i)
            qc.rx(2*t, i)
        if op_str.count('Z') == 2:
            i, j = i_j
            qc.cx(i, j)
            qc.rz(2*s*op_coeff, j)
            qc.cx(i, j)
            qc.h(i)
            qc.rx(np.pi/2, i)
            qc.rzx(2*t, i, j)
            qc.rx(-np.pi/2, i)
            qc.h(i)
    final = QuantumCircuit(n)
    for rep in range(reps):
        t_p = Parameter('t_{}'.format(rep))
        s_p = Parameter('s_{}'.format(rep))
        final.append(qc.assign_parameters({t:t_p, s:s_p}), range(n))
#     print(qc.draw(fold=200))
#     print(final.draw(fold=200))
    return final

def construct_initial_state(no_cars, no_routes):
    """[summary]

    Args:
        args ([type]): [description]

    Returns:
        [type]: [description]
    """
    R = no_routes
    N = no_cars
    #Using gates
    w_one = QuantumCircuit(R)
    for r in range(0,R-1):
        if r == 0:
            w_one.ry(2*np.arccos(1/np.sqrt(R-r)),r)
        elif r != R-1:
            w_one.cry(2*np.arccos(1/np.sqrt(R-r)), r-1, r)
    for r in range(1,R):
        w_one.cx(R-r-1,R-r)
    w_one.x(0)

    # #Using Initial Instruction
    # w_one = QuantumCircuit(R)
    # probs = [0 for _ in range(2**R)]
    # for i in range(R):
    #     probs[2**i] += 1/np.sqrt(R)
    # w_one.initialize(probs)

    #Tensor product for N cars
    initial_state = w_one.copy()
    for _ in range(0,N-1):
        initial_state = initial_state.tensor(w_one)
    return initial_state, w_one

In [27]:
qc = ansatz(9, 1, operator)

In [28]:
basis_gates = ["id", "h", "rx", "ry", "rz", "cx"]
qc = transpile(qc, basis_gates = basis_gates)

In [23]:
qc.draw(fold=200)

┌─────────────────────┐┌───────────┐                                                                                                                                                       »
q_0: ┤ RZ(-10.7180801*s_0) ├┤ RX(2*t_0) ├───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────»
     ├─────────────────────┤├───────────┤                                                                                                                                                       »
q_1: ┤ RZ(-10.1509701*s_0) ├┤ RX(2*t_0) ├───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────»
     ├─────────────────────┤├───────────┤                                                                                                                                                       »
q_2: ┤ RZ(-10.7447961*s_0) ├┤ RX(2*t_0) ├───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────»
     └┬────────────────────┤├───────────┤                                                                                                                                                       »
q_3: ─┤ RZ(-9.3006891*s_0) ├┤ RX(2*t_0) ├───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────»
      ├────────────────────┤├───────────┤                                                                                                                                                       »
q_4: ─┤ RZ(-8.7335791*s_0) ├┤ RX(2*t_0) ├───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────»
      ├────────────────────┤├───────────┤                                                                                                                                                       »
q_5: ─┤ RZ(-9.3274051*s_0) ├┤ RX(2*t_0) ├───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────»
      ├────────────────────┤├───────────┤                                                                                                              ┌───┐┌───────────────────┐┌───┐┌───┐┌───┐»
q_6: ─┤ RZ(-4.7389551*s_0) ├┤ RX(2*t_0) ├──────────────────────────────────────────────────────────────────────────────────────────────────────────────┤ X ├┤ RZ(4.1281391*s_0) ├┤ X ├┤ H ├┤ X ├»
      ├────────────────────┤├───────────┤┌───┐┌───────────────────┐┌───┐┌───┐┌───┐┌───────────┐┌───┐┌───┐┌─────────┐┌───┐┌───────────┐┌───┐┌──────────┐└─┬─┘└───────────────────┘└─┬─┘└───┘└─┬─┘»
q_7: ─┤ RZ(-5.6541281*s_0) ├┤ RX(2*t_0) ├┤ X ├┤ RZ(4.3987191*s_0) ├┤ X ├┤ H ├┤ X ├┤ RZ(2*t_0) ├┤ X ├┤ H ├┤ RX(π/2) ├┤ X ├┤ RZ(2*t_0) ├┤ X ├┤ RX(-π/2) ├──┼─────────────────────────┼─────────┼──»
      ├────────────────────┤├───────────┤└─┬─┘└───────────────────┘└─┬─┘├───┤└─┬─┘└───────────┘└─┬─┘├───┤├─────────┤└─┬─┘└───────────┘└─┬─┘├──────────┤  │                         │  ┌───┐  │  »
q_8: ─┤ RZ(-5.8095051*s_0) ├┤ RX(2*t_0) ├──■─────────────────────────■──┤ H ├──■─────────────────■──┤ H ├┤ RX(π/2) ├──■─────────────────■──┤ RX(-π/2) ├──■─────────────────────────■──┤ H ├──■──»
      └────────────────────┘└───────────┘                               └───┘                       └───┘└─────────┘                       └──────────┘                               └───┘     »
«                                                                                                                                                                                                     »
«q_0: ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [62]:
from qiskit.circuit.library import QAOAAnsatz

In [3]:
import dlib

In [9]:
asd = dlib.find_min_global

In [10]:
asd()

TypeError: find_min_global(): incompatible function arguments. The following argument types are supported:
    1. (f: object, bound1: list, bound2: list, is_integer_variable: list, num_function_calls: int, solver_epsilon: float=0) -> tuple
    2. (f: object, bound1: list, bound2: list, num_function_calls: int, solver_epsilon: float=0) -> tuple

Invoked with: 

In [11]:
nlopt_minimize

NameError: name 'nlopt_minimize' is not defined

In [28]:
opt = nlopt.opt(nlopt.LN_SBPLX, 2)

In [29]:
opt.__dir__()

['this',
 '__module__',
 'thisown',
 '__repr__',
 '__swig_destroy__',
 '__init__',
 'optimize',
 'last_optimize_result',
 'last_optimum_value',
 'get_algorithm',
 'get_algorithm_name',
 'get_dimension',
 'set_min_objective',
 'set_max_objective',
 'remove_inequality_constraints',
 'remove_equality_constraints',
 'add_inequality_constraint',
 'add_equality_constraint',
 'add_inequality_mconstraint',
 'add_equality_mconstraint',
 'set_param',
 'get_param',
 'has_param',
 'nth_param',
 'num_params',
 'get_lower_bounds',
 'set_lower_bounds',
 'get_upper_bounds',
 'set_upper_bounds',
 'get_stopval',
 'set_stopval',
 'get_ftol_rel',
 'set_ftol_rel',
 'get_ftol_abs',
 'set_ftol_abs',
 'get_xtol_rel',
 'set_xtol_rel',
 'get_xtol_abs',
 'set_xtol_abs',
 'get_x_weights',
 'set_x_weights',
 'get_maxeval',
 'set_maxeval',
 'get_numevals',
 'get_maxtime',
 'set_maxtime',
 'get_force_stop',
 'set_force_stop',
 'force_stop',
 'get_errmsg',
 'set_local_optimizer',
 'get_population',
 'set_population',